<a href="https://colab.research.google.com/github/Gouthammajjari/Intro_to_ml/blob/main/assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.1 MB/s eta 0:00:00
time: 265 µs (started: 2024-05-04 02:25:55 +00:00)


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

time: 6.8 s (started: 2024-05-04 02:25:57 +00:00)


In [ ]:
torch.manual_seed(123)

time: 6.71 ms (started: 2024-05-04 02:26:03 +00:00)


In [ ]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

100%|██████████| 170498071/170498071 [00:04<00:00, 42609007.83it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
time: 16.1 s (started: 2024-05-04 02:26:03 +00:00)


In [ ]:
mean

tensor([0.4914, 0.4822, 0.4465])

time: 37.4 ms (started: 2024-05-04 02:26:20 +00:00)


In [ ]:
std

tensor([0.2470, 0.2435, 0.2616])

time: 3.16 ms (started: 2024-05-04 02:26:20 +00:00)


In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

time: 52.2 ms (started: 2024-05-04 02:26:20 +00:00)


In [ ]:
device

device(type='cuda')

time: 2.38 ms (started: 2024-05-04 02:26:20 +00:00)


In [ ]:
# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
])

time: 645 µs (started: 2024-05-04 02:26:20 +00:00)


In [ ]:
cifar10 = datasets.CIFAR10(
    './data', train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


time: 489 ms (started: 2024-05-04 02:26:20 +00:00)


In [ ]:
cifar10_val = datasets.CIFAR10(
     './data', train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

time: 379 ms (started: 2024-05-04 02:26:20 +00:00)


In [ ]:
first_image, label = cifar10[0]
print(first_image.shape)

torch.Size([3, 32, 32])
time: 7.12 ms (started: 2024-05-04 02:26:21 +00:00)


In [ ]:
train_loader = DataLoader(cifar10, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(cifar10_val, batch_size=32, shuffle=False, num_workers=2)

time: 638 µs (started: 2024-05-04 02:26:21 +00:00)


In [ ]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.Tanh(),
    nn.Linear(512, 10)
).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

time: 155 ms (started: 2024-05-04 02:26:21 +00:00)


In [ ]:
def train_model(model, train_loader, test_loader, num_epochs=300, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Testing the model
        model.eval()
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = correct / total

        if epoch % 1 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Test Accuracy: {accuracy * 100:.2f}%')

    # Classification Report
    report = classification_report(all_labels, all_predicted)
    print("Classification Report:\n", report)


time: 1.01 ms (started: 2024-05-04 02:26:21 +00:00)


In [ ]:
train_model(model, train_loader, test_loader, num_epochs=100, lr=0.01)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1/100, Loss: 1.7894863032104873, Test Accuracy: 41.73%
Epoch 2/100, Loss: 1.6517056608840737, Test Accuracy: 42.54%
Epoch 3/100, Loss: 1.5802918802219863, Test Accuracy: 44.33%
Epoch 4/100, Loss: 1.517758316476606, Test Accuracy: 45.54%
Epoch 5/100, Loss: 1.4601751830207181, Test Accuracy: 45.93%
Epoch 6/100, Loss: 1.4057868315253743, Test Accuracy: 47.12%
Epoch 7/100, Loss: 1.3519397465677805, Test Accuracy: 46.98%
Epoch 8/100, Loss: 1.2997348973252265, Test Accuracy: 48.17%
Epoch 9/100, Loss: 1.2462299844048683, Test Accuracy: 47.50%
Epoch 10/100, Loss: 1.1947670439764695, Test Accuracy: 48.68%
Epoch 11/100, Loss: 1.142497247102851, Test Accuracy: 47.90%
Epoch 12/100, Loss: 1.0925753005635486, Test Accuracy: 48.14%
Epoch 13/100, Loss: 1.0409287122755728, Test Accuracy: 48.22%
Epoch 14/100, Loss: 0.9911015565320611, Test Accuracy: 48.42%
Epoch 15/100, Loss: 0.9411930450627382, Test Accuracy: 47.76%
Epoch 16/100, Loss: 0.89478384010775, Test Accuracy: 48.22%
Epoch 17/100, Loss: 0

In [ ]:
model2 = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.Tanh(),
    nn.Linear(512, 256),
    nn.Tanh(),
    nn.Linear(256, 128),
    nn.Tanh(),
    nn.Linear(128, 10)
).to(device)

time: 17.4 ms (started: 2024-05-04 02:26:34 +00:00)


In [ ]:
train_model(model2, train_loader, test_loader, num_epochs=300, lr=0.01)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1/300, Loss: 1.8558505601175153, Test Accuracy: 39.45%
Epoch 2/300, Loss: 1.690991407011231, Test Accuracy: 42.14%
Epoch 3/300, Loss: 1.6219095023717167, Test Accuracy: 43.75%
Epoch 4/300, Loss: 1.5618384869069681, Test Accuracy: 44.91%
Epoch 5/300, Loss: 1.5054588581001003, Test Accuracy: 46.76%
Epoch 6/300, Loss: 1.4501071113542494, Test Accuracy: 47.13%
Epoch 7/300, Loss: 1.393197675279067, Test Accuracy: 47.28%
Epoch 8/300, Loss: 1.3340140860277494, Test Accuracy: 48.18%
Epoch 9/300, Loss: 1.2791666343893977, Test Accuracy: 48.68%
Epoch 10/300, Loss: 1.2184916274408766, Test Accuracy: 45.78%
Epoch 11/300, Loss: 1.1621051681628993, Test Accuracy: 49.30%
Epoch 12/300, Loss: 1.1008202402696958, Test Accuracy: 47.51%
Epoch 13/300, Loss: 1.0398080933193175, Test Accuracy: 49.00%
Epoch 14/300, Loss: 0.9811650291895607, Test Accuracy: 47.63%
Epoch 15/300, Loss: 0.916386367625635, Test Accuracy: 47.34%
Epoch 16/300, Loss: 0.8549687948008797, Test Accuracy: 45.03%
Epoch 17/300, Loss: 

In [ ]:
total_params = sum(p.numel() for p in model2.parameters() if p.requires_grad)
print("Total number of trainable parameters ", total_params,  '\n')

Total number of trainable parameters  1738890 

time: 923 µs (started: 2024-05-04 03:12:31 +00:00)
